# This file is for creating the benchmark, with stacked frames.

# import relevant packages

In [1]:
import os
import datetime
import wandb

from stable_baselines3 import PPO
from stable_baselines3.common.callbacks import CallbackList, EvalCallback
from stable_baselines3.common.env_util import make_atari_env
from stable_baselines3.common.vec_env import VecFrameStack, VecTransposeImage, VecVideoRecorder

from wandb.integration.sb3 import WandbCallback

from feature_extraction.callbacks.wandb_on_training_end_callback import WandbOnTrainingEndCallback
from utils import linear_schedule
from collections import OrderedDict

2024-03-30 22:13:32.853999: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 22:13:32.854162: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 22:13:32.894304: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-30 22:13:33.033158: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-30 22:13:34.772505: W tensorflow/compiler/tf2

# Settings

In [2]:
progress_bar = True
train_model = True
eval_model = False
project_name = "atari"
#Human readable timestamp
timestamp = datetime.datetime.now().strftime('_%Y-%m-%d_%H-%M-%S')

run_name = "breakout_benchmark" + timestamp
log_dir = "logs"


# Setup Wandb

In [3]:

wandb.login()

config = OrderedDict([
    # Environment settings
    ('env_id', "ALE/Breakout-v5"),
    ('n_envs', 8),
    ('env_wrapper', ['stable_baselines3.common.atari_wrappers.AtariWrapper']),
    ('frame_stack', 4),
    ('training_seed', 12),
    ('evaluation_seed', 14),
    
    # Algorithm and policy
    ('algo', 'PPO'),
    ('policy', 'CnnPolicy'),
    
    # Training hyperparameters
    ('batch_size', 256),
    ('n_steps', 128),
    ('n_epochs', 4),
    ('n_timesteps', 3_000),
    ('learning_rate', 0.00025),
    ('learning_rate_schedule', 'linear'),
    ('clip_range', 0.1),
    ('clip_range_schedule', 'linear'),
    ('ent_coef', 0.01),
    ('vf_coef', 0.5),
    ('normalize_advantage', False),
    
    # Evaluation and logging
    ('n_eval_episodes', 2),
    ('record_n_episodes', 10),
    ('log_frequency', 1024),
    
    # Other settings
    ('verbose', 1)
])
root_logdir = os.getcwd()
#wandb.tensorboard.patch(root_logdir=root_logdir)

wandb.init(
    project=project_name,
    name=run_name, # Name of the run
    config=config,
    save_code=True,
    sync_tensorboard=True,
    #monitor_gym=True,
)

config = wandb.config

wandb: Currently logged in as: adicreson (feature_extraction). Use `wandb login --relogin` to force relogin


# Create Evaluation Environment

In [4]:
vec_eval_env = make_atari_env(config.env_id, n_envs=config.n_envs, seed=config.evaluation_seed)
vec_eval_env = VecFrameStack(vec_eval_env, n_stack=config.frame_stack)
vec_eval_env = VecTransposeImage(vec_eval_env)


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


# Create Training Environment

In [5]:
vec_train_env = make_atari_env(config.env_id, n_envs=config.n_envs, seed=config.training_seed)
vec_train_env = VecFrameStack(vec_train_env, n_stack=config.frame_stack)
vec_train_env = VecTransposeImage(vec_train_env)

# Create Model

In [6]:
# Define the keys for PPO-specific hyperparameters
ppo_params_keys = [
    'batch_size',
    'ent_coef',
    'n_epochs',
    'n_steps',
    'policy',
    'vf_coef',
    'normalize_advantage',
]

# Filter the config dictionary to extract only the PPO hyperparameters
ppo_hyperparams = {key: config[key] for key in ppo_params_keys if key in config}

# Additional hyperparameters not in the initial filter that require custom handling
learning_rate_schedule = linear_schedule(2.5e-4)
clip_range_schedule = linear_schedule(0.1)

# Instantiate the PPO model with the specified hyperparameters and environment
model = PPO(
    **ppo_hyperparams,
    learning_rate=learning_rate_schedule,
    clip_range=clip_range_schedule,
    env=vec_train_env, 
    verbose=1,
    tensorboard_log=f"{log_dir}",
)


Using cpu device


/home/maliti/Prog/Global-Feature-Extraction/venv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


# Create Callbacks

In [7]:
# Save best model
eval_callback = EvalCallback(
    eval_env=vec_eval_env,
    eval_freq=max(config.log_frequency // config.n_envs, 1),
    n_eval_episodes=config.n_eval_episodes,
    best_model_save_path=log_dir,
    log_path=log_dir,
    deterministic=True,
    render=False,
    verbose=0
)
 

# Needs to be changed, so it uses run instead of wandb
wandb_callback = WandbCallback(
    verbose=1,
    gradient_save_freq=256,
)

wandb_on_training_end_callback = WandbOnTrainingEndCallback(
    model=model,
    eval_env=vec_eval_env,
    log_dir=log_dir,
    n_eval_episodes=config.n_eval_episodes,
    record_n_episodes=config.record_n_episodes,
)
callbacks = CallbackList([wandb_callback, eval_callback, wandb_on_training_end_callback])

# Train Model with callbacks

In [8]:
model.learn(
    total_timesteps=config.n_timesteps,
    callback=callbacks,
)       

Logging to logs/PPO_1


2024-03-30 22:13:43.787699: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-30 22:13:43.787746: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-30 22:13:43.788828: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-30 22:13:43.796794: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-30 22:13:44.658329: W tensorflow/compiler/tf2

---------------------------------
| eval/              |          |
|    mean_ep_length  | 168      |
|    mean_reward     | 0        |
| time/              |          |
|    total_timesteps | 1024     |
---------------------------------
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 238      |
|    ep_rew_mean     | 1.86     |
| time/              |          |
|    fps             | 152      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 1024     |
---------------------------------
-------------------------------------------
| eval/                   |               |
|    mean_ep_length       | 280           |
|    mean_reward          | 3             |
| time/                   |               |
|    total_timesteps      | 2048          |
| train/                  |               |
|    approx_kl            | 0.00082574395 |
|    clip_fraction        | 0.0383        |
|    clip_range           

/home/maliti/Prog/Global-Feature-Extraction/venv/lib/python3.10/site-packages/stable_baselines3/common/distributions.py:87: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if deterministic:


Evaluating model...
Mean reward: 6.00 +/- 2.00
Model evaluation done.
Logging model architecture...
Model Summary: ==========================================================================================
Layer (type:depth-idx)                   Output Shape              Param #
ActorCriticCnnPolicy                     [1]                       --
├─NatureCNN: 1-1                         [1, 512]                  --
│    └─Sequential: 2-1                   [1, 3136]                 --
│    │    └─Conv2d: 3-1                  [1, 32, 20, 20]           8,224
│    │    └─ReLU: 3-2                    [1, 32, 20, 20]           --
│    │    └─Conv2d: 3-3                  [1, 64, 9, 9]             32,832
│    │    └─ReLU: 3-4                    [1, 64, 9, 9]             --
│    │    └─Conv2d: 3-5                  [1, 64, 7, 7]             36,928
│    │    └─ReLU: 3-6                    [1, 64, 7, 7]             --
│    │    └─Flatten: 3-7                 [1, 3136]                 --
│    └─S

# Cleanup

In [9]:
wandb.finish()

best_model/mean_reward,▁
best_model/std_reward,▁
eval/mean_ep_length,▁▅█
eval/mean_reward,▁▆█
global_step,▁▅█
rollout/ep_len_mean,█▂▁
rollout/ep_rew_mean,█▂▁
time/fps,█▅▁
train/approx_kl,█▁
train/clip_fraction,█▁
train/clip_range,█▁
